In [2]:
import pandas as pd
from graph_tool import Graph
from netpixi.data.gt import *
import netpixi
from graph_tool import centrality
import seaborn as sns
import matplotlib.pyplot as plt
from graph_tool import clustering
import cpnet
import statsmodels.formula.api as smf
from graph_tool import spectral
from matplotlib import rcParams
import numpy as np
from math import sqrt
import csv

# Importando o Dataset

In [2]:
!pip install opendatasets

     |████████████████████████████████| 97 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 149 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 10.3 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=d4cf1190865cdd035acc6fe34b9e29cd0dfb784a62b6c88d31fffd523f82e38d
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
import opendatasets as od

In [4]:
od.download("https://www.kaggle.com/stackoverflow/stacksample")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: beatrizmuniz
Your Kaggle Key: ········


100%|██████████| 1.11G/1.11G [03:19<00:00, 5.97MB/s]


# Lendo os Dados

In [46]:
tags = pd.read_csv("stacksample/Tags.csv")
quests = pd.read_csv("stacksample/Questions.csv", encoding='ISO-8859-1', engine='python')
answ = pd.read_csv("stacksample/Answers.csv", encoding='ISO-8859-1', engine='python')

In [49]:
#filtrando perguntas apenas para aquelas de 2016 
quests = quests[(quests['CreationDate'] >= '2016-00-00T00:00:00Z')]

In [50]:
#numero total de perguntas em 2016
quests.shape[0]

211049

In [51]:
#filtrando as tags para pegar apenas as dos posts de 2016
tags = tags[tags['Id'].isin(quests['Id'])]

In [53]:
#quantidade de tags no total (não únicas; cada fileira representa uma tag associada a uma pergunta;uma tag pode ser 
#associada a mais de uma pergunta)
tags.shape[0]

631280

In [54]:
#filtrando as tags para manter apenas as perguntas que possuem duas ou mais tags associadas
tags = tags.groupby('Id').filter(lambda x: len(x) > 2)

In [55]:
#quantidade de tags depois de filtrar
tags.shape[0]

498815

In [56]:
#filtrando as perguntas para ter apenas as com 2 ou mais tags
quests = quests[quests['Id'].isin(tags['Id'])]

In [57]:
#quantidade de perguntas depois de filtrar
quests.shape[0]

132407

# Fazendo o grafo

In [69]:
#setup do grafo
g = Graph(directed=False) # não dirigido
g.vp['name'] = g.new_vp('object')
g.ep['counter'] = g.new_ep('object')

In [72]:
# adiciona vertice de uma tag ao grafo
def addTag(tag):
    g.add_vertex()
    list_tags.append(tag)
    v_tag = g.vertex(list_tags.index(tag))
    g.vp['name'][v_tag] = str(tag)

In [73]:
#adiciona aresta ao grafo quando duas tags estão presentes na mesma pergunta
def addConnec(tag1, tag2):
    index1 = list_tags.index(tag1)
    index2 = list_tags.index(tag2)
    
    if g.edge(index1, index2) == None:
        g.add_edge(index1, index2)
        e=g.edge(index1, index2)
        g.ep['counter'][e] = 1
    else:
        e=g.edge(index1, index2)
        g.ep['counter'][e] = g.ep['counter'][e] + 1

In [82]:
list_tags = []
for tagIndex in range(tags.shape[0] - 2):
    tag = tags.iloc[tagIndex]['Tag']
    questId = tags.iloc[tagIndex]['Id']
    
    sameQuest = True
    i=1
    
    while sameQuest:
        nextTag = tags.iloc[tagIndex + i]['Tag']
        nextId = tags.iloc[tagIndex + i]['Id']

        if questId == nextId:
            if tag not in list_tags:
                addTag(tag)
            if nextTag not in list_tags:
                addTag(nextTag)

            addConnec(tag, nextTag)
        else:
            sameQuest = False
        
        if ((tagIndex + i) < (tags.shape[0] - 1)):
            i += 1

KeyboardInterrupt: 

In [75]:
#posicionando vertices de acordo com o algoritmo SFDP, que evidencia "panelinhas"
m = draw_gt.sfdp_layout(g)
move_gt(g, m)
save_gt(g, 'exemplo.net.gz')

In [78]:
#rederiza visualizacao do grafo
r = netpixi.render('exemplo.net.gz')

In [77]:
#mostra os nomes dos jogadores na visualizacao
#CUIDADO QUE QUANDO VC RODA ISSO O ARQUIVO FICA INFINTAMENTE MAIS PESADO
r.vertex_set_key('name')

In [83]:
#quantidade de tags únicas
len(list_tags)

22258